In [7]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
import pandas as pd
import model 
from flower_dataset import FlowerDataset
from trainer_mixup_arcface import Trainer
from config import *
from model_zoo import get_model

In [8]:
## dataset & dataloader
df_train_label = pd.read_csv('train_label.csv')
df_test_label = pd.read_csv('test_label.csv')

# dataset
train_dataset = FlowerDataset('./training/', df_train_label, image_transform)
test_dataset = FlowerDataset('./training/', df_test_label, image_transform)

# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
train_dataset[0][0].shape

torch.Size([3, 448, 448])

In [10]:
# custom layer 
if is_arcface: 
    custom_layer = model.Arcface_Mixup(class_num, embed_size) if is_mixup else model.Arcface(class_num, embed_size)
else: 
    custom_layer = model.FFN(embed_size, class_num)

# final model 
final_model = model.Model(get_model(model_name, embed_size, drop_p), custom_layer).to(device)

In [ ]:
%%time
train_pipeline = Trainer()
train_pipeline.compile(lr, decay_fn, loss_fn, metric_dict, is_arcface, is_mixup, device=device)
train_pipeline.fit(final_model, train_dataloader, test_dataloader, num_epoch, save_config)

100%|██████████| 55/55 [00:57<00:00,  1.04s/it, loss=5.29]


train_loss: 5.416900591218853
{'Accuracy': 0.1392693966627121, 'CE': 5.09618616104126}


 96%|█████████▋| 53/55 [00:54<00:02,  1.03s/it, loss=5.05]